### Case Verbatim Extractor
*Author: Justin Ho*

The more I program, the more I realize that the naming is the most difficult thing to do rings truer and truer as the day goes by. But enough with my complaints! This ipynb takes in the html files from the Case Dockets, and take the contents of the important parts of the case, hence **CaseVerbatim**.

In [1]:
from bs4 import BeautifulSoup
import os
import sqlite3

## Conncect to database
con = sqlite3.connect(os.environ["database_name"])
cur = con.cursor()

In [2]:
existing_cases = [c[0] for c in cur.execute("SELECT CaseID from CaseVerbatim;").fetchall()]

In [3]:
## Check all of the Case Dockets that were scraped
files = os.listdir("CaseRecords")

In [ ]:
## iterate through all of the files
for file in files:
    filename = f"./CaseRecords/{file}"

    # Read the contents of the HTML file
    with open(filename, "r", encoding="utf-8") as f:
        html_content = f.read()
        
    # Parse the HTML
    soup = BeautifulSoup(html_content, "html.parser")
    panels = soup.find_all("div", {"class": "panel panel-default"})

    ## Initialize with empty string
    case_summary = ""
    parties = ""
    offense = ""
    actions = ""

    ## Go through each important panel
    for panel in panels:

        if panel.find("h3").text == "Case Summary":
            case_summary = panel.find("pre").text

        if panel.find("h3").text == "Parties/Attorneys to the Case":
            parties = panel.find("pre").text

        if panel.find("h3").text == "Offense Information":
            offense = panel.find("pre").text
            
        if panel.find("h3").text == "Register of Actions":
            actions = panel.find("pre")


    ## For SQLITE purposes
    case_id = file.replace(".html", "")
    case_summary = case_summary.replace("\"", "\'")
    parties = parties.replace("\"", "\'")
    offense = offense.replace("\"", "\'")
    actions = str(actions).replace("\"", "\'")

    if case_id in existing_cases:
        
        cur.execute(f"""
            UPDATE CaseVerbatim
            SET CaseSummary = \"{case_summary}\",
            Parties = \"{parties}\",
            OffenseInformation = \"{offense}\",
            RegisterOfActions = \"{actions}\"
            WHERE CaseID = \"{case_id}\"
        """)

    else:
    
        ## Insert!
        cur.execute(f"""
                    INSERT INTO CaseVerbatim 
                    (CaseID, CaseSummary, Parties, OffenseInformation, RegisterOfActions) 
                    VALUES (\"{case_id}\", \"{case_summary}\", \"{parties}\", \"{offense}\", \"{actions}\")
                """)
    
    
    con.commit()